<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/1_download_videos_from_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube==12.1.0

In [ ]:
#imports
from pytube import YouTube
import os
import shutil

In [ ]:
#config
PROJECT = 'rc_car_tracking'
COLAB_BASE_PROJECT = f'/content/{PROJECT}'

DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}'
DRIVE_VIDEOS_FOLDER = 'videos'

In [ ]:
#google drive
from google.colab import drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)

In [ ]:
#symbolic link
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
#remove and create OUTPUT_VIDEOS_FOLDER folder
shutil.rmtree(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}', ignore_errors=True)
os.makedirs(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}', exist_ok=True)

In [ ]:
#download videos from YouTube at maximum resolution
video_links = [
  "https://www.youtube.com/watch?v=CwfQ-c7if90", #train
  "https://www.youtube.com/watch?v=gXZeDXmwSQU", #train (to ffmpeg)
  "https://www.youtube.com/watch?v=J1DoMRO0ShY", #train
  "https://www.youtube.com/watch?v=s64_GIoMEmg", #test
  "https://www.youtube.com/watch?v=sHYisA3dCNU" #test
]

for i,video_link in enumerate(video_links):
  print(f"Downloading video {i+1} of {len(video_links)}...")
  yt = YouTube(video_link)
  yt.streams.filter(file_extension='mp4').order_by('resolution')[-1].download(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}', f'{i}_{"train" if i < (len(video_links) - 2) else "test"}.mp4')

print('Job done!')

In [ ]:
#ffmpeg to convert video '2018 IFMAR ISTC World Championships - A-main Leg 1.mp4' from AV1 to mp4 codec
!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod 755 ./ffmpeg-colab/install
!./ffmpeg-colab/install
!rm -rf ./ffmpeg-colab
!ffmpeg -version
!ffmpeg -y -i '$COLAB_BASE_PROJECT/$DRIVE_VIDEOS_FOLDER/1_train.mp4' '$COLAB_BASE_PROJECT/$DRIVE_VIDEOS_FOLDER/converted.mp4'
!rm '$COLAB_BASE_PROJECT/$DRIVE_VIDEOS_FOLDER/1_train.mp4'
!mv '$COLAB_BASE_PROJECT/$DRIVE_VIDEOS_FOLDER/converted.mp4' '$COLAB_BASE_PROJECT/$DRIVE_VIDEOS_FOLDER/1_train.mp4'

In [ ]:
#create short video portions useful for the test
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

for video in os.listdir(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}'):
  ffmpeg_extract_subclip(f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}/{video}', 120, 150, targetname=f'{COLAB_BASE_PROJECT}/{DRIVE_VIDEOS_FOLDER}/{os.path.splitext(video)[0]}_short.mp4')